# Working with Scientific Datasets

<img src="images/000_000_epom_logo.png" alt="ePOM" title="" align="center" width="12%" alt="Python logo\"></a>

---

The purpose of this notebook is to demonstrate the use of a few related standards and libraries related to working with oceanographic datasets. This involves:
1. Learning about Xarray data structures and plotting
2. Saving and importing NetCDF files
3. Importing TableDAP datasets

Before you start this notebook, you should make sure you have completed the prior numpy notebooks.

## 1 Introduction to Xarray

The NumPy package introduced in earlier notebooks was useful for manipulating data held in numerical arrays. Another packge Xarray builds upon NumPy to add more convenience features and make the arrays more self-descriptive for use as scientific datasets. Xarray documentation can be found here: https://docs.xarray.dev/en/stable/user-guide/index.html 

First, import some modules we will need later.

In [ ]:
from pydap.client import open_url
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import os

%load_ext autoreload
%autoreload 2
%matplotlib inline

<img align="left" width="6%" style="padding-right:10px;" src="images/key.png">

In Xarray, we can now store metadata in our arrays, such as names for our array dimensions, and names for each index along each axis. We can still use numpy arrays and features on xarray data.

---

### 1.1 Xarray DataArray

Let's have a look at the xarray datatypes and how we can use them. We'll start with the `DataArray`, which stores a numpy array with dimension names and coordinate labels (a name for each index we would use to access a particular part of an array)

To demonstrate, here's an Xarray `DataArray` with two named dimensions


In [ ]:
my_map = xr.DataArray(
    #setup the multidimensional array's data
    data=np.random.randn(5,5),
    #set the names of each dimension of our array
    dims=["latitude","longitude"],
    # give some names to different indices of our arrays, as we feel like it
    coords={
		"latitude":[0,5,15,30,90], #coordinates must be in ascending order, but don't have to be linear
		"longitude":[0,5,15,30,90], #we could also use string names such as "apple"
	}, 
	name="my_data_array",
	attrs={
    	"some":"metadata", # any metadata of your choice
    	"units":"degrees C", #what does the value at some coordinate represent? (used by xarray plots)
    	"long_name":"Global Temperature" #Title of your array (used by xarray)
	}
)
my_map.latitude.attrs["units"] = "degrees" #set metadata for our axis / coordinates
my_map.longitude.attrs["units"] = "degrees" #set metadata for our axis / coordinates

my_map

Observe that printing the `DataArray` shows us the data, attributes (metadata), and coordinates. You can click the page and database (cylinder) icons under coordinates to see the attributes and data of the coordinate arrays associated with our main data.

xarray also incorporates MatPlotLib, and will use the metadata associated with our array to setup some matplotlib or cartopy data automatically.

In [ ]:
my_map.plot()
plt.title(my_map.long_name)
plt.show()

As you can see, xarray can plot our data using the metadata we gave it when we constructed the array. The latitude axis was scaled according to fit the numeric coordinate labels. There are a couple restrictions on the coordinates, namely:
- The number of coordinates must match the length of the corresponding dimension, or be omitted for that dimension
- To plot, numeric coordinates must be sorted (you will get an error if you try the sequence `5,50,10,20,30` as the coordinate labels for one of the axis above for example)

<img align="left" width="6%" style="padding-right:10px;" src="images/test.png">

Setup your own `DataArray` and make a plot with it just like we demonstrated above. This time, make it a 10 by 10 grid and use evenly spaced coordinate labels. Label the y coordinate as depth in meters, and the x coordinate as time. Keep the values of the `DataArray` as temperature.

I suggest using NumPy's linspace to generate evenly spaced numbers, and np arrange, datetime and timedelta to generate the date labels.

Solution:

In [ ]:
dateArray_sol = np.arange(
    np.datetime64("2025-10-01"), 
    np.datetime64("2025-10-11"), 
    np.timedelta64(24,"h")
)

a1_sol = xr.DataArray(
    data=np.random.randn(10,10),
    dims=["depth","time"],
    coords={
		"depth":np.linspace(0,9,10),
		"time": dateArray_sol
	}, 
	name="my_data_array",
	attrs={
    	"some":"metadata", 
    	"units":"degrees C",
    	"long_name":"Ocean Temperature pver time"
	}
)
a1_sol.depth.attrs["units"] = "m"

a1_sol.plot(yincrease=False) #we can use this to invert the y-axis of the plot
plt.title(a1_sol.long_name)

---

## 1.2 Basic Xarray data selection

Xarray can do selections and operations using combinations of dimension labels and coordinates. Let's go over a few basic examples of these.

In [ ]:
#sel selects by coordinate name (5,15,30,90 earlier). This example uses it to select a single value
my_map.sel({"latitude":30, "longitude":90})

In [ ]:
#isel selects by dimension name and array index. This example uses it to select a numpy array subset of a dataarray
my_map.isel(longitude=2)

In [ ]:
#filter array by coordinate value
my_map.where(my_map.latitude==30)

Computation and selection should also work the same as Numpy, except that we can use dimension and coordinate names instead of indices if we like.

In [ ]:
#take the median along the longitude axis, for each latitude
my_map.median("longitude")

<img align="left" width="6%" style="padding-right:10px;" src="images/test.png">

Take the average (mean) of my_map for every latitude. Then select the value where longitude is `15`.

In [ ]:
#Solution
my_map.mean("latitude").sel(longitude=5)

---

## 1.3 Xarray Datasets

In addition to the `DataArray` class, Xarray provides the `Dataset` class, which is a dictionary of `DataArray`s. If we put multiple `DataArray`s with matching some axis into a `Dataset`, then we can work with these DataArrays together. Sharing the axis is not required to put the `DataArrays` into the `Dataset`, but it is useful.

In [ ]:
my_dataset = xr.Dataset(
    {
    	"Temperature":my_map, 
    	"Salinity":(["lat","lon","depth"],np.random.randn(5,5,3)), #create a new dataarray by specifying coordinate, then data
		"Oxygen":(["lat","lon","depth"],np.random.randn(5,5,3))
 	},
    {
		"depth":[0,50,100] # we can label dimensions with coordinates here
	},
    {
		"some":"more metadata in attrs"
	}
)
my_dataset.depth.attrs["units"] = "m"
my_dataset

We cannot to a line plot on a `Dataset`, but we can do a scatter plot, among a few other options

In [ ]:
my_dataset.plot.scatter(x="lat", y="depth", z="lon")

---

The last important Xarray datastructure for this lab is the `DataTree`.
Each `DataTree` can have one parent data tree, and 0 to many child `DataTree`s, their data is contained in a `Dataset`. The point of this is that if we organize the data we generate into a single tree structure, we can then save the tree to a file.

In [ ]:
# Create some datasets
ds1 = xr.Dataset(
    {
		"temperature":(["x","y","z"],np.random.randn(5,5,3))
	},
)
ds2 = xr.Dataset(
    {
		"salinity":(["x","y","z"],np.random.randn(5,5,3))
	},
)
ds3 = xr.Dataset(
    {
		"conductivity":(["x","y","z"],np.random.randn(5,5,3))
	},
)
ds_root = xr.Dataset(
    {
		#"casts":(["station"],[1])
	},
)


#create the DataTree 
leaf1 = xr.DataTree(ds1, name="temp")
leaf2 = xr.DataTree(ds2, name="sal")
leaf3 = xr.DataTree(ds3, name="cond")
root = xr.DataTree(ds_root, {"temp":leaf1,"sal":leaf2,"leaf3":leaf3}, name="root")
root


<img align="left" width="6%" style="padding-right:10px;" src="images/test.png">

Using the existing `ds1` through `ds_root` datasets, create a new `DataTree` where temperature and salinity are both children of conductivity, and conductivity is a child of the root node. Examine how the tree is different by expanding the groups tab when the `DataTree` is printed.

In [ ]:
#Solution
leaf1_sol = xr.DataTree(ds1, name="temp")
leaf2_sol = xr.DataTree(ds2, name="sal")
cond_sol = xr.DataTree(ds3, name="cond", children={"temp":leaf1,"salt":leaf2})
root_sol = xr.DataTree(ds_root, {"cond":cond_sol}, name="root_sol")
root_sol

---

# 2. NetCDF

NetCDF is a file format used in the ocean and atmospheric studies to hold data. In its current version it is built on top of the HDF5 format. Xarray was designed so that its concepts (Metadata storage along with DataArrays, data organized in a tree format with DataTree) would line up almost 1 to 1 with NetCDF. This means that reading and writing NetCDF files only takes one function call with Xarray.

Here's how to save a file to NetCDF

In [ ]:
file_name = "./my_dataset.nc"
if os.path.exists(file_name):
    os.remove(file_name)
root_sol.to_netcdf(file_name)
root_sol.close()

We can also import the same file, observe that the printed DataTree is the same as before.

In [ ]:
reimport = xr.open_datatree("./my_dataset.nc")
reimport

Be sure to close your files when done with them! Otherwise you may get a resource lock error when trying to modify or delete the file with another program.

In [ ]:
reimport.close()

---

# 3. OPeNDAP

OPeNDAP is a way of accessing datasets provided by certain institutions such as NOAA and NASA. It uses HTTP under the hood, and lets you access data stored on servers in NetCDF and HDF formats. Xarray of course lets us access NetCDF datasets on the web easily through OPeNDAP.

We will use a dataset from Norway.
1. Go to this website. https://catalog-intaros.nersc.no/dataset/ctd-data-collected-with-r-v-hakon-mosby
2. ![Go to the resource on the 2002 dataset](images/Norway1.png) Go to the resource on the 2002 dataset
3. ![Then copy the OPENDAP link](images/Norway2.png) Then copy the OPENDAP link by right clicking then using "copy clean link"
4. use `xr.open_dataset(url)` to open the NetCDF file, you will need to remove the `.html` at the end of the link.

In [ ]:
norway = xr.open_dataset("https://opendap1.nodc.no/thredds/dodsC/physics/point/yearly/58AA_CTD_2002.nc")
norway

We can see that this dataset has time, latitude, longitude as coordinates, and its variables also have position and depth coordinates.

<img align="left" width="6%" style="padding-right:10px;" src="images/test.png">

Try using xarray and the norway dataset to make a plot.

Sample solution

In [ ]:

norway.PSAL.attrs["units"] = "psu"
norway.PSAL.isel(TIME=slice(0,5)).plot.line(y="DEPTH", yincrease=False)
plt.title("CTD Cast Salinity from 2002")

<img align="left" width="6%" style="padding-right:10px;" src="images/info.png">

Checkout this websites for more about netCDF files
- https://adyork.github.io/python-oceanography-lesson/17-Intro-NetCDF/index.html

You can also try downloading some NetCDF files from one of these websites:
- https://doi.pangaea.de/10.1594/PANGAEA.880029
- https://catalog-intaros.nersc.no/dataset/ctd-data-collected-with-r-v-hakon-mosby
- https://data.gov.au/data/dataset/rv-investigator-voyage-in2017_t02-ctd-data
- https://www.ncei.noaa.gov/access/metadata/landing-page/bin/iso?id=gov.noaa.nodc:DFO-Profile
- https://www.marine.csiro.au/data/trawler/download.cfm?file_id=3685
- https://data.ceda.ac.uk/bodc/gebco/global
- https://ncss.hycom.org/thredds/catalog.html
- https://www.ncei.noaa.gov/access/metadata/landing-page/bin/iso?id=gov.noaa.nodc:NCEI-WOD

---

# 3.1 TABLEDAP

We another typical datasource are tabledap datasets. This is a service for tabular datasets built on OPeNDAP and ERDDAP. Various organizations have made data available through TABLEDAP services, for example:
- https://erddap.ifremer.fr/erddap/tabledap/index.html?page=1&itemsPerPage=1000
- https://www.ncei.noaa.gov/erddap/tabledap/index.html?page=1&itemsPerPage=1000
- https://www.smartatlantic.ca/erddap/tabledap/index.html?page=1&itemsPerPage=1000
- https://dap.oceannetworks.ca/erddap/tabledap/index.html?page=1&itemsPerPage=1000
- https://erddap.ogsl.ca/erddap/tabledap/index.html?page=1&itemsPerPage=1000

For this tutorial, we will use the smartatlantic tabledap services.

1. First, go to one of those links, and click on a "data" link in the `DAP Data` column for a dataset that sounds interesting.
2. You will be presented with a "Data Access Form" page which will helpfully show you the various variables this dataset contains. Copy the url to this page, which will be similar to "https://www.smartatlantic.ca/erddap/tabledap/SMA_saint_john.html", place it in a python variable
3. Remove the ".html" from the url.
4. Use `xr.open_dataset(url)` to open the dataset


In [ ]:
xr_tabledap_url ="https://www.smartatlantic.ca/erddap/tabledap/SMA_saint_john"
SJ_dap = xr.open_dataset(xr_tabledap_url)
SJ_dap

In [ ]:
SJ_dap["s.wind_chill"].attrs

TableDAP Datasets can be awkward to work with, so here is an example of making a new DataArray from the previous data. Notice how we use the rename function to map the "s" coordinate (row number) to a new "time" coordinate for better labelling.

In [ ]:

SJ_arr = xr.DataArray(
    data=SJ_dap["s.air_pressure_avg"],
    coords={"time":SJ_dap["s.time"].rename({"s":"time"})},
    dims="time"
)
SJ_arr

In [ ]:
SJ_arr.plot()

---

There is another module called `PyDAP` we can use to look at `OPeNDAP` datasets more quickly. We will need to further modify the url from before:
1. replace the https protocol with "dap2" and 
2. open the dataset with open_url from the pydap module we imported earlier.

In [ ]:
url_pydap = "dap2://www.smartatlantic.ca/erddap/tabledap/SMA_saint_john"
pydap_ds = open_url(url_pydap)

We can get a succinct tree view from the pydap module with the `.tree()` function.

In [ ]:
pydap_ds.tree()

We can view the attributes of the dataset with pydap too with a slightly different access call.

In [ ]:
pydap_ds['s']['curr_spd_avg'].attributes

In this tutorial we only scratched the surface of Xarray, NetCDF, and PyDAP, but hopefully this introduction will help you as you progress in ocean data science. Be sure to take a look at the xarray user guide and other documentation for more information and tutorials!

---

 ## <image align="left" width="6%" style="padding-right:10px;" src="images/refs.png"> Useful References and Information


- https://tutorial.xarray.dev/intro.
- https://docs.xarray.dev/en/stable/user-guide/io.html
- https://pydap.github.io/pydap/en/5_minute_tutorial.html
- https://notebook.community/alaindomissy/xarray_example/Exploring%20netCDF%20Datasets%20Using%20xarray
- https://carpentries-lab.github.io/python-aos-lesson/02-visualisation.html
- https://lijodxl.github.io/OceanographyWithPython/L4-working_with_netCDF.html
- https://geohackweek.github.io/nDarrays/
- https://github.com/hmedrano/erddap-python
- https://opendap.github.io/documentation/
- https://www.ncei.noaa.gov/erddap/tabledap/documentation.html